In [1]:
import numpy as np
import pandas as pd
import os
import ast
import nltk
import functools
import re
import json
import statistics
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

In [30]:
WORD_COUNTS_NAME = 'word_counts'
WORD_COUNTS_LIST_NAME = 'word_counts_list'
TOTAL_NAME = 'total_words'
COMMENTS_COLUMN = 'comments'
FREQUENCY_NAME = 'frequency'
MAGNIFY_LIST = ['too', 'very', 'that', 'so', 'as']
NEGATIVE_LIST = ['not', 'aren\'t', 'isn\'t', 'wasn\'t', 'didn\'t']
with open('constants/word_categories.json', 'r') as f:
    WORD_CATEGORIES = json.load(f)
IMPORTANT_WORDS = reduce(lambda x, y: x + y, WORD_CATEGORIES.values())

In [3]:
# HIGH MEMORY: Don't load unless you have to
with open('data/overall_list.json', 'r') as f:
    ALL_WORKLOAD_LIST = json.load(f)
    
# TODO: Finish this for other metrics
# with open('overall_list.json', 'r') as f:
#     ALL_RECOMMEND_LIST = json.load(f)

In [5]:
'''Fixes the fact that some zeros are encoded as empty list'''
def fix_list(x):
    try:
        return int(x)
    except:
        return 0
    
df = pd.read_csv('raw_data/final.csv')
df.enrollment = df.enrollment.apply(fix_list)

In [19]:
'''Takes a body of text and returns a list of words in that text.'''
def get_words(text):
    return re.compile('\w+').findall(text)

'''See if a word was used in a negative context'''
def probe_if_negative(word1, word2):
    if word1 in NEGATIVE_LIST:
        return True
    elif (word2 in NEGATIVE_LIST) and (word1 in MAGNIFY_LIST):
        return True
    else:
        return False

def get_word_dict(comments, to_return):
    d = {}
    t = 0
    for comment in comments:
        last_word = ''
        two_words_ago = ''
        for word in get_words(comment):
            word = word.lower() # Make lower case
            t += 1
            try:
                if probe_if_negative(last_word, two_words_ago):
                    d['not_' + word] += 1
                else:
                    d[word] += 1
            except: 
                if probe_if_negative(last_word, two_words_ago):
                    d['not_' + word] = 1
                else:
                    d[word] = 1
            two_words_ago = last_word
            last_word = word
    if to_return == 'words':
        return d
    elif to_return == 'count':
        return t

def preprocessing(to_return, row):
    raw_comments = row[COMMENTS_COLUMN]
    return get_word_dict(ast.literal_eval(raw_comments), to_return)

def count_words(in_df):
    in_df = in_df.copy()
    in_df[TOTAL_NAME] = in_df.apply(functools.partial(preprocessing, 'count'), axis=1)
    in_df[WORD_COUNTS_NAME] = in_df.apply(functools.partial(preprocessing, 'words'), axis=1)
    return in_df

'''Calculates the frequency of a word in the comments of row'''
def calculate_frequency(word, row):
    try:
        return float(row[WORD_COUNTS_NAME][word]) / float(row[TOTAL_NAME])
    except:
        return 0
    
'''Weighted mean'''
def weighted_mean(x, w):
    return np.sum(x * w) / np.sum(w)

'''Weighted covariance'''
def weighted_cov(x, y, w):
    return np.sum(w * (x - weighted_mean(x, w)) * (y - weighted_mean(y, w))) / np.sum(w)

'''Weighted correlation'''
def corr(x, y, w):
    return weighted_cov(x, y, w) / np.sqrt(weighted_cov(x, x, w) * weighted_cov(y, y, w))

''' Returns two column dataframe consisting of df[column] and the frequency of the word.
    Weights according to the number of comments. '''
def find_word_correlations(in_df, word, column):
    freq = in_df.apply(functools.partial(calculate_frequency, word), axis=1)
    possibly_nan = pd.concat([freq, in_df[column], in_df[COMMENTS_COLUMN]], 
                             axis=1, 
                             keys=[FREQUENCY_NAME, column, COMMENTS_COLUMN])
    clean_df = possibly_nan.dropna(axis=0, how='any')
    return corr(clean_df[FREQUENCY_NAME], 
                clean_df[column],
                clean_df[COMMENTS_COLUMN].apply(lambda x: len(ast.literal_eval(x))))

In [20]:
get_words("There was a man who wasn't dead")

['There', 'was', 'a', 'man', 'who', 'wasn', 't', 'dead']

In [93]:
def sum_dict_values(some_dict):
    return sum(some_dict.values())

'''Finds overall frequency of words in df by constructing total count and total dict
    in_df: The input df which expects to ??
    add_method: if counter, ??? if list, ???
'''
def find_word_freqs_over_df(in_df, agg_method):
    in_df = in_df.copy()
    if agg_method == 'counter':
        D = Counter({})
        for index, row in in_df.iterrows():
            D += Counter(row[WORD_COUNTS_NAME])
        return dict(D)
    elif agg_method == 'list':
        D = {}
        keys = set({})
        for index, row in in_df.iterrows():
            r = row[WORD_COUNTS_LIST_NAME]
            keys = keys.union(set(r.keys()))
            for key in keys:
                try:
                    Dval = D[key]
                except:
                    Dval = []
                try:
                    rval = r[key]
                except:
                    rval = []
                D[key] = Dval + rval
        return D
    
'''Breaks df into subdfs, then finds frequencies for subdfs.'''
def find_group_word_freqs(whole_df, gb):
    whole_df = whole_df.copy()
    gb = whole_df.groupby(gb)    
    return [(x, find_word_freqs_over_df(gb.get_group(x), "counter")) for x in gb.groups]

'''Turns a dict of occurence count form into a dict of target_value_list form'''
def convert_dict(in_dict, value):
    in_dict = dict(in_dict) # Make a copy so nothing bad happens
    new_dict = {}
    for k, v in in_dict.iteritems():
        new_dict[k] = [value] * v
    return new_dict

'''Applies convert dict to each row of the in_df with values according to column'''
def apply_list_conversion(in_df, column):
    in_df = in_df.copy() # Copy so nothing bad happens
    in_df[WORD_COUNTS_LIST_NAME] = (in_df[[WORD_COUNTS_NAME, column]]
                                    .apply(lambda x: convert_dict(x[WORD_COUNTS_NAME], x[column]), axis=1))
    return in_df

'''Finds a specific group of words.  
    aggregate_list: Given some rating, this is the list for every word's data.  
    word_list: the specific words cared about.  '''
def find_specific_word_values(aggregate_list, word_list):
    D = {}
    for word in word_list:
        D[word] = [x for x in aggregate_list[word] if not np.isnan(x)]
    return D

'''Truncates to most frequent 50 words in each dict.  Puts dicts in '''
def package_word_freqs(freqs_list_tuples, excluded_words):
    output = {}
    for category_name, freqs_list in freqs_list_tuples:
        for grouping, freqs in freqs_list:
            name = create_group_name(grouping)
            freq_list_some = straighten_list(freqs, 100, excluded_words) # nothing excluded
            freq_list_all = straighten_list(freqs, 100, []) # nothing excluded
            print freq_list_some
            try:
                output[category_name][name] = {"all": freq_list_all, "interesting": freq_list_some}
            except:
                output[category_name] = {}
                output[category_name][name] = {"all": freq_list_all, "interesting": freq_list_some}
    return output

def create_group_name(tuple_or_string):
    if type(tuple_or_string) == tuple:
        return tuple_or_string[1] + '_' + str(tuple_or_string[0])
    else:
        return tuple_or_string

'''Turns dict k, v set into list and truncates to truncate value.'''
def straighten_list(in_dict, truncate_value, excluded_words):
    lst = sorted(in_dict.items(), key=lambda word_count: -word_count[1])
    straightened = filter(lambda x: (x[0] not in excluded_words), lst)
    print(len(lst), len(straightened))
    return straightened[:truncate_value]

In [9]:
df = pd.read_csv("raw_data/final.csv")
df2 = count_words(df)

In [14]:
df3 = apply_list_conversion(df2, 'Course_Workload_Rating')
df4 = apply_list_conversion(df2, 'Course_Overall_Rating')

In [88]:
with open('cdf_data.json', 'w') as outfile:
        json.dump(cdf_data, outfile)

with open('data/all_workload_list.json', 'w') as outfile:
        json.dump(ALL_WORKLOAD_LIST, outfile)

### Getting Data for Visualization of Frequency Plots

In [17]:
# Will get a dictionary featuring the counts of every word.
overall_workload = find_word_freqs_over_df(df3, 'counter')
overall_rating = find_word_freqs_over_df(df4, 'counter')

In [25]:
# Will get a dictionary featuring 
'''e.g. find_word_freqs_over_df(df3, 'list') = {
    word1: [workload_use1, workload_use2, ...],
    word2: [workload_use1], 
    ...
}'''
occurrences_workload = find_word_freqs_over_df(df3, 'list')
occurrences_rating = find_word_freqs_over_df(df4, 'list')

In [33]:
with open('raw_data/overall_workload.json', 'w') as outfile:
    json.dump(overall_workload, outfile)
with open('raw_data/overall_rating.json', 'w') as outfile:
    json.dump(overall_rating, outfile)
with open('raw_data/occurrences_workload.json', 'w') as outfile:
    json.dump(occurrences_workload, outfile)
with open('raw_data/occurrences_rating.json', 'w') as outfile:
    json.dump(occurrences_rating, outfile)

In [32]:
vis_workload = find_specific_word_values(occurrences_workload, IMPORTANT_WORDS)
vis_rating = find_specific_word_values(occurrences_rating, IMPORTANT_WORDS)
with open('data/short_workload_list.json', 'w') as outfile:
    json.dump(vis_workload, outfile)
with open('data/short_overall_list.json', 'w') as outfile:
    json.dump(vis_rating, outfile)

### Preparing for Word Cloud

For each department and for each term, we want to produce two lists: 
    - A list of the 100 most common words in that department/term's comments with the #occurrences
    - Same thing but filter out the boring words (and, or, not, a, I, you, ...)
The output data format for this will be in the form 
output_json = {
    dept_or_term1: {
        all: [[top_word, num1], [next_word, num2], ...],
        interesting: [[top_interesting_word, new_num1], [next_interesting_word, new_num2], ...]
    },
    ...
}

In [36]:
# All words (boring included)
by_department = find_group_word_freqs(df2, 'department1')
by_year = find_group_word_freqs(df2, ['year', 'term'])

In [94]:
with open('constants/boring_words.json', 'r') as f:
    BORING_WORDS = json.load(f)

In [95]:
wordcloud_data = package_word_freqs([('department', by_department), ('year', by_year)], BORING_WORDS)

(563, 518)
(563, 563)
[('class', 65), ('swedish', 44), ('language', 33), ('fun', 28), ('great', 22), ('about', 18), ('learn', 14), ('course', 14), ('agnes', 14), ('get', 14), ('take', 12), ('more', 10), ('time', 10), ('culture', 10), ('also', 9), ('want', 8), ('amazing', 8), ('think', 8), ('classes', 8), ('best', 8), ('there', 8), ('interesting', 7), ('better', 7), ('wonderful', 7), ('she', 7), ('where', 7), ('can', 6), ('help', 6), ('one', 6), ('many', 6), ('learning', 6), ('read', 6), ('like', 6), ('ursula', 6), ('ever', 5), ('teacher', 5), ('from', 5), ('work', 5), ('grammar', 5), ('going', 5), ('learned', 5), ('easy', 5), ('everything', 5), ('way', 5), ('made', 5), ('would', 4), ('few', 4), ('me', 4), ('awesome', 4), ('how', 4), ('ones', 4), ('them', 4), ('good', 4), ('day', 4), ('didn', 4), ('put', 4), ('final', 4), ('need', 4), ('any', 4), ('requirement', 4), ('lots', 4), ('harvard', 4), ('make', 4), ('well', 4), ('just', 4), ('has', 4), ('know', 4), ('own', 4), ('her', 4), ('tau

[('course', 35), ('class', 34), ('papers', 22), ('great', 14), ('interesting', 14), ('read', 14), ('material', 11), ('genomics', 10), ('want', 10), ('from', 9), ('more', 9), ('learn', 9), ('reading', 9), ('biology', 9), ('there', 9), ('professors', 8), ('research', 8), ('topics', 8), ('field', 8), ('excellent', 7), ('paper', 7), ('take', 7), ('get', 7), ('interested', 7), ('worth', 6), ('assignments', 6), ('useful', 6), ('work', 6), ('good', 6), ('discussions', 6), ('however', 6), ('can', 6), ('up', 6), ('learned', 5), ('would', 5), ('quite', 5), ('put', 5), ('lectures', 5), ('current', 5), ('only', 5), ('taking', 5), ('overview', 5), ('general', 5), ('different', 5), ('quantitative', 5), ('about', 5), ('time', 5), ('what', 4), ('topic', 4), ('way', 4), ('its', 4), ('before', 4), ('them', 4), ('each', 4), ('em', 4), ('out', 4), ('one', 4), ('population', 4), ('classes', 4), ('many', 4), ('learning', 4), ('week', 4), ('without', 4), ('just', 4), ('background', 4), ('some', 4), ('omics',

(1140, 1140)
[('class', 147), ('great', 88), ('course', 75), ('professor', 72), ('interesting', 69), ('take', 54), ('about', 49), ('fun', 48), ('work', 41), ('material', 41), ('history', 40), ('celtic', 37), ('mckenna', 30), ('assignments', 28), ('get', 27), ('would', 26), ('do', 25), ('readings', 25), ('can', 24), ('irish', 24), ('harvard', 23), ('interested', 22), ('learn', 22), ('one', 21), ('workload', 21), ('reading', 20), ('wonderful', 19), ('also', 19), ('definitely', 19), ('enjoyable', 19), ('lectures', 19), ('just', 18), ('easy', 18), ('stories', 18), ('fantastic', 17), ('recommend', 17), ('culture', 17), ('any', 17), ('what', 16), ('more', 16), ('fascinating', 16), ('who', 16), ('manageable', 15), ('amazing', 14), ('too', 14), ('ireland', 14), ('people', 14), ('time', 14), ('there', 14), ('even', 13), ('good', 13), ('engaging', 13), ('literature', 13), ('from', 13), ('well', 13), ('medieval', 13), ('study', 12), ('how', 12), ('most', 12), ('subject', 12), ('read', 12), ('take

[('course', 5), ('from', 3), ('utopian', 3), ('perspective', 2), ('opportunity', 2), ('interdisciplinary', 2), ('material', 2), ('field', 2), ('history', 2), ('students', 2), ('other', 2), ('literature', 2), ('studies', 2), ('painting', 1), ('poetry', 1), ('simple', 1), ('major', 1), ('one', 1), ('professors', 1), ('semester', 1), ('range', 1), ('quality', 1), ('little', 1), ('religion', 1), ('utopias', 1), ('encompasses', 1), ('methodology', 1), ('fiction', 1), ('communities', 1), ('interested', 1), ('taken', 1), ('john', 1), ('start', 1), ('archival', 1), ('more', 1), ('do', 1), ('good', 1), ('photography', 1), ('about', 1), ('cultural', 1), ('e', 1), ('edifying', 1), ('apart', 1), ('dystopian', 1), ('departments', 1), ('up', 1), ('try', 1), ('stauffer', 1), ('civ', 1), ('where', 1), ('trips', 1), ('dan', 1), ('challenging', 1), ('concord', 1), ('am', 1), ('high', 1), ('pass', 1), ('not_want', 1), ('take', 1), ('film', 1), ('seminars', 1), ('broad', 1), ('discussion', 1), ('when', 1)

(14, 8)
(14, 14)
[('great', 1), ('one', 1), ('wishes', 1), ('take', 1), ('armenian', 1), ('learn', 1), ('class', 1), ('speak', 1)]
(0, 0)
(0, 0)
[]
(1910, 1862)
(1910, 1910)
[('class', 220), ('course', 176), ('professor', 99), ('take', 98), ('great', 93), ('about', 89), ('east', 57), ('get', 57), ('interesting', 53), ('week', 52), ('writing', 50), ('also', 46), ('from', 45), ('readings', 44), ('time', 43), ('do', 42), ('more', 41), ('asian', 40), ('can', 40), ('papers', 39), ('like', 39), ('there', 38), ('topics', 36), ('one', 36), ('learn', 35), ('interested', 35), ('think', 34), ('fun', 33), ('just', 33), ('eas', 33), ('what', 32), ('want', 32), ('harvard', 32), ('work', 32), ('would', 31), ('good', 31), ('life', 31), ('way', 31), ('reading', 31), ('students', 31), ('me', 29), ('section', 28), ('each', 28), ('abe', 27), ('japan', 27), ('which', 27), ('response', 26), ('some', 26), ('by', 26), ('studies', 26), ('amazing', 25), ('paper', 25), ('most', 25), ('material', 25), ('tf', 24),

(315, 278)
(315, 315)
[('immunology', 20), ('course', 19), ('class', 11), ('great', 8), ('field', 7), ('interesting', 7), ('research', 6), ('learn', 5), ('background', 5), ('good', 5), ('current', 4), ('can', 4), ('discussion', 4), ('well', 4), ('being', 4), ('professors', 4), ('questions', 4), ('about', 4), ('lectures', 4), ('do', 3), ('overall', 3), ('cancer', 3), ('overview', 3), ('from', 3), ('science', 3), ('work', 3), ('may', 3), ('data', 3), ('each', 3), ('by', 3), ('their', 3), ('able', 3), ('papers', 3), ('most', 3), ('having', 3), ('enjoyed', 2), ('state', 2), ('only', 2), ('outside', 2), ('familiar', 2), ('discussions', 2), ('techniques', 2), ('said', 2), ('review', 2), ('various', 2), ('reading', 2), ('many', 2), ('studies', 2), ('useful', 2), ('those', 2), ('experts', 2), ('allowed', 2), ('subjects', 2), ('different', 2), ('get', 2), ('amazing', 2), ('practice', 2), ('had', 2), ('material', 2), ('using', 2), ('day', 2), ('some', 2), ('taught', 2), ('looking', 2), ('provide

In [96]:
with open('data/wordcloud_data.json', 'w') as outfile:
    json.dump(wordcloud_data, outfile)

### Preparing data for Enrollment Data

In [22]:
# USEFUL FUNCTIONS FOR THIS PART
def filter_nans(val):
    if val == [] or val == '[]':
        return 0
    else:
        return float(val)

In [23]:
df = pd.read_csv('raw_data/final.csv')

In [97]:
of_interest = df[['department1', 'year', 'term', 
                  'enrollment', 'name_key1', 'course_title', 'Course_Workload_Rating', 'Course_Overall_Rating']]
of_interest.loc[:, 'enrollment'] = (of_interest['enrollment']
                                     .apply(lambda x: filter_nans(x)))

In [98]:
top_depts = (of_interest.groupby('department1')['enrollment'].sum())
top_depts_list = top_depts.astype('int').sort_values().index[-30:].tolist()

In [99]:
of_more_interest = (of_interest[of_interest['department1']
                                .isin(top_depts_list)])

In [100]:
of_more_interest.to_csv('enrollment.csv')

In [101]:
of_more_interest.head()

,department1,year,term,enrollment,name_key1,course_title,Course_Workload_Rating,Course_Overall_Rating
0,AESTHINT,2011,fall,59.0,AESTHINT 13,Cultural Agents,1.5,3.7
1,AESTHINT,2011,fall,48.0,AESTHINT 15,Elements of Rhetoric,2.3,4.2
2,AESTHINT,2011,fall,111.0,AESTHINT 24,First Nights: Five Performance Premieres,1.5,3.9
3,AESTHINT,2011,fall,99.0,AESTHINT 26,"Race, Gender, and Performance",1.6,3.8
4,AESTHINT,2011,fall,29.0,AESTHINT 30,Love In A Dead Language: Classical Indian Lite...,2.0,4.3


In [102]:
# Enrollment by semester
# [{year: 2011_spring, MATH: 10, ...}]
of_more_interest['year_term'] = of_more_interest['year'].astype(str)+ '_' + of_more_interest['term']
info = of_more_interest.groupby(['year_term', 'department1'])['enrollment'].sum().to_frame().reset_index()
enrollment_history = []
for time in info.year_term.unique():
    d = {}
    d['year'] = time
    for _idx, row in info[info.year_term == time].iterrows():
        d[row['department1']] = row['enrollment']
    enrollment_history = enrollment_history + [d]
with open('data/enrollment_history.json', 'w') as outfile:
        json.dump(enrollment_history, outfile)

/usr/local/Cellar/ipython@5/5.5.0_4/libexec/vendor/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### For Computing Median Student

In [107]:
def compute_one_median(vals):
    cutoff = sum(vals) / 2
    start = 0
    lastval = vals[0]
    for val in vals:
        start += val
        if start >= cutoff:
            return (val + lastval) / 2

def compute_median_student_dict(df):
    d = {}
    for department in df['department1'].unique():
        gf = df[df['department1'] == department]
        vals = gf.enrollment.tolist()
        d[department] = compute_one_median(vals)
    return d

def compute_median_dict(df):
    d = {}
    for department in df['department1'].unique():
        gf = df[df['department1'] == department]
        vals = gf.enrollment.tolist()
        d[department] = statistics.median(vals)
    return d

In [124]:
median_student = compute_median_student_dict(of_more_interest)
median = compute_median_dict(of_more_interest)
with open("data/median_student.json", 'w') as outfile:
    json.dump(median_student, outfile)
with open("data/median.json", 'w') as outfile:
    json.dump(median, outfile)

### For Computing Correlations between workload and rating.

In [119]:
def compute_correlation_dict(df):
    d = {}
    df = df.dropna(0)
    for department in df['department1'].unique():
        gf = df[df['department1'] == department]
        w = np.array(gf.enrollment)
        x = np.array(gf.Course_Workload_Rating.map(float))
        y = np.array(gf.Course_Overall_Rating.map(float))
        d[department] = corr(x,y,w)
    return d

In [123]:
correlations = compute_correlation_dict(of_more_interest)
with open("data/correlations.json", 'w') as outfile:
    json.dump(correlations, outfile)

### For difficulty aggregate

In [156]:
relevant_cols = ['Course_Workload_Distribution', 
                 'Course_Overall_Distribution',
                 'Course_Overall_Respondents',
                 'Course_Workload_Rating',
                 'Course_Overall_Rating',
                 'Course_Workload_Respondents',
                 'enrollment', 
                 'name_key1',
                 'course_title', 
                 'year',
                 'term',
                 'department1']

In [157]:
CONVENTIONS = {
    'workload': {
        'distribution': 'Course_Workload_Distribution',
        'mean': 'Course_Workload_Rating',
        'intervals': [{'min': 0, 'max': 3},
                      {'min': 4, 'max': 7},
                      {'min': 8, 'max': 11},
                      {'min': 11, 'max': 14},
                      {'min': 11, 'max': 17}],
        'std_multiplier': 3,
        'respondents': 'Course_Workload_Respondents'
    },
    'overall': {
        'distribution': 'Course_Overall_Distribution',
        'mean': 'Course_Overall_Rating',
        'intervals': [{'min': 1, 'max': 1},
                      {'min': 2, 'max': 2},
                      {'min': 3, 'max': 3},
                      {'min': 4, 'max': 4},
                      {'min': 5, 'max': 5}],
        'std_multiplier': 1,
        'respondents': 'Course_Overall_Respondents'
    }
}

In [210]:
def get_ith_list_val(i, lst):
    if type(lst) == type(np.nan):
        return 0
    else:
        lst = ast.literal_eval(lst)
        if len(lst) == 0:
            return 0
        else:
            return lst[i]
        
def get_list_stat_nice(lst, func, std_multiplier):
    big_list = []
    for i in range(len(lst)):
        big_list += ([i] * lst[i])
    if len(big_list) == 1 and func == statistics.stdev: # Need 2+ data pts for variance
        return -1
    if func == statistics.stdev:
        return func(big_list) * std_multiplier
    else:
        return func(big_list)

def get_list_stat(lst, func, std_multiplier):
    if type(lst) == type(np.nan):
        return -1
    else:
        lst = ast.literal_eval(lst)
        if len(lst) == 0 or reduce(lambda x, y: x+y, lst) == 0:
            return -1
        else:
            return get_list_stat_nice(lst, func, std_multiplier)

def get_stat_score(statistic, desired, lst):
    if desired == 'median':
        intervals = CONVENTIONS[statistic]['intervals']
        m = get_list_stat(lst, statistics.median, 1)
        if m == -1:
            return np.nan
        if float(m).is_integer():
            i = int(m)
            return 0.5 * (intervals[i]['max'] + intervals[i]['min'])
        else:
            m = int(m-0.5)
            return 0.5 * (intervals[m]['max'] + intervals[m + 1]['min'])
    elif desired == 'stdev':
        std_multiplier = CONVENTIONS[statistic]['std_multiplier']
        return get_list_stat(lst, statistics.stdev, std_multiplier)

def add_distribution_data(in_df, statistics):
    in_df = in_df.copy()
    for statistic in statistics:
        column = CONVENTIONS[statistic]['distribution']
        for i in range(5):
            in_df[statistic + '_s' + str(i+1)] = (in_df[column]
                .apply(functools.partial(get_ith_list_val, i)))
    return in_df

def add_mean_data(in_df, statistics):
    in_df = in_df.copy()
    for statistic in statistics:
        in_df[statistic + '_mean'] = in_df[CONVENTIONS[statistic]['mean']]
    return in_df

def add_respondents_data(in_df, statistics):
    in_df = in_df.copy()
    for statistic in statistics:
        in_df[statistic + '_respondents'] = in_df[CONVENTIONS[statistic]['respondents']]
    return in_df

def add_instdev_data(in_df, statistics):
    in_df = in_df.copy()
    for statistic in statistics:
        column = CONVENTIONS[statistic]['distribution']
        in_df[statistic + '_instdev'] = (in_df[column]
                                        .apply(functools.partial(get_stat_score, statistic, 'stdev')))
    return in_df


def add_additional_data(in_df, statistics):
    in_df = in_df.copy()
    in_df = add_respondents_data(in_df, statistics)
    in_df = add_distribution_data(in_df, statistics)
    in_df = add_mean_data(in_df, statistics)
    in_df = add_instdev_data(in_df, statistics)
    return in_df

'''Computes weighted mean'''
def compute_grouped_mean(in_df, val_col, weight_col):
    in_df = in_df.copy()
    return weighted_mean(x=in_df[val_col], w=in_df[weight_col])

'''Computes the inter-group std (std of means weighted according to num respondants)'''
def compute_grouped_outstd(in_df, mean_col, weight_col):
    in_df = in_df.copy()
    return np.sqrt(weighted_cov(w=in_df[weight_col], x=in_df[mean_col], y=in_df[mean_col]))

def add_grouped_data(in_df, gb, statistics):
    in_df = in_df.copy()
    distribution_columns = []
    for statistic in statistics:
        for num in [1, 2, 3, 4, 5]:
            distribution_columns += [statistic + '_s' + str(num)]
    gp_df = in_df.groupby(gb)[distribution_columns].sum()
    for statistic in statistics:
        gp_df = (gp_df.join(in_df.groupby(gb)[[statistic + '_mean', statistic + '_respondents']]
                           .apply(compute_grouped_mean, statistic + '_mean', statistic + '_respondents')
                           .to_frame(statistic + '_mean'))
                 .join(in_df.groupby(gb)[[statistic + '_instdev', statistic + '_respondents']]
                       .apply(compute_grouped_mean, statistic + '_instdev', statistic + '_respondents')
                       .to_frame(statistic + '_instdev'))
                 .join(in_df.groupby(gb)[[statistic + '_mean', statistic + '_respondents']]
                       .apply(compute_grouped_outstd, statistic + '_mean', statistic + '_respondents')
                       .to_frame(statistic + '_outstdev')))
    return gp_df

def make_all_stats_for_year(in_df, year):
    in_df = in_df.copy()
    relevant_df = in_df[in_df.year == year][RELEVANT_COLS]
    basic_stats_df = add_additional_data(relevant_df, ['workload', 'overall'])
    grouped_stats_df = add_grouped_data(basic_stats_df, 'department1', ['workload', 'overall'])
    grouped_stats_df = grouped_stats_df[grouped_stats_df.index.isin(top_depts_list)]
    return grouped_stats_df

In [211]:
basic_stats_df = add_additional_data(relevant_df, ['workload', 'overall'])
grouped_stats_df = add_grouped_data(basic_stats_df, 'department1', ['workload', 'overall'])
grouped_stats_df = grouped_stats_df[grouped_stats_df.index.isin(top_depts_list)]

In [214]:
grouped_stats_df.sort_values('overall_outstdev')

,workload_s1,workload_s2,workload_s3,workload_s4,workload_s5,overall_s1,overall_s2,overall_s3,overall_s4,overall_s5,workload_mean,workload_instdev,workload_outstdev,overall_mean,overall_instdev,overall_outstdev
department1,,,,,,,,,,,,,,,,
SPANSH,1332,4042,999,112,50,51,207,972,2541,2987,3.191905,1.944804,1.579686,4.214117,0.795844,0.282742
ETHRSON,2055,3438,588,46,30,102,408,1379,2583,2024,2.847669,1.888124,1.224740,3.925062,0.903916,0.306533
CULTBLF,3270,4356,666,62,23,102,456,1722,3472,3096,2.468545,1.837499,1.053884,4.015269,0.863717,0.313039
PHYSCI,195,2167,1739,362,153,167,593,1511,1641,825,4.810962,2.368908,2.268167,3.500401,0.973573,0.333174
SOCWORLD,1606,3413,849,112,29,110,358,1245,2379,2204,2.816991,1.951843,1.156378,3.992122,0.895333,0.356579
SCILIVSY,1864,2815,523,56,29,138,378,1300,1980,1690,2.673917,2.025728,1.134484,3.850583,0.941988,0.371696
STAT,1302,5254,3712,956,822,231,847,2782,4712,4199,4.811647,2.269072,3.149624,3.927641,0.900646,0.383429
EXPOS,1015,6102,2275,311,134,134,582,1800,3756,3873,3.725099,2.041996,1.676108,4.049660,0.864011,0.390902
ENGLISH,852,4593,2105,373,150,74,189,926,2343,4896,3.755890,1.983221,1.989326,4.400688,0.701565,0.391545


## For Listing Within Department

This will allow the sorting of particular classes.

In [68]:
ranked = df[['term', 'year', 'name_key1', 'department1', 'enrollment', 'course_title', 'Course_Workload_Rating', 'Course_Overall_Rating']]
ranked['Course'] = ranked["name_key1"].map(str) + ': ' + ranked["course_title"]
ranked = ranked.drop(['name_key1', 'course_title'], 1)
ranked = ranked.rename(columns={'Course_Workload_Rating': 'Workload',
                                'Course_Overall_Rating': 'Overall',
                                'enrollment': 'Enrollment',
                                'department1': 'department'}).set_index('Course')
ranked.to_csv('data/ranked.csv')

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [69]:
ranked

,term,year,department,Enrollment,Workload,Overall
Course,,,,,,
AESTHINT 13: Cultural Agents,fall,2011,AESTHINT,59,1.5,3.7
AESTHINT 15: Elements of Rhetoric,fall,2011,AESTHINT,48,2.3,4.2
AESTHINT 24: First Nights: Five Performance Premieres,fall,2011,AESTHINT,111,1.5,3.9
"AESTHINT 26: Race, Gender, and Performance",fall,2011,AESTHINT,99,1.6,3.8
AESTHINT 30: Love In A Dead Language: Classical Indian Literature and Its Theorists,fall,2011,AESTHINT,29,2.0,4.3
AESTHINT 33: Ancient Fictions: The Ancient Novel in Context,fall,2011,AESTHINT,81,1.7,4.1
AESTHINT 35: Forms in Korean Cultural History,fall,2011,AESTHINT,9,1.6,3.9
AESTHINT 37: Introduction to the Bible in the Humanities and the Arts,fall,2011,AESTHINT,182,1.8,3.1
AESTHINT 38: The English Language as Literature,fall,2011,AESTHINT,16,1.6,4.1
